In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
df=pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
df

In [3]:
df.info()

In [4]:
df.describe()

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke Pie Chart</h1>


In [5]:

labels =df['stroke'].value_counts(sort = True).index
sizes = df['stroke'].value_counts(sort = True)

colors = ["lightblue","red"]
explode = (0.05,0) 
 
plt.figure(figsize=(7,7))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=90,)

plt.title('Customer Churn Breakdown')
plt.show()

* Only 5% percent of people have Stroke ! 


<h1 style="font-family:Georgia;font-size:175%;text-align:left">Gender</h1>


In [6]:
plt.figure(figsize=(10,5))
sns.countplot(data=df,x='gender');

* There is about 1000 diffrence between Female and Male in the data 


<h1 style="font-family:Georgia;font-size:175%;text-align:left">Correlation with avg glucose level</h1>

* Visualize some features which maybe have correlation with avg glucose level



In [7]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
df.plot(kind='scatter', x='age', y='avg_glucose_level', alpha=0.5, color='green', ax=axes[0], title="Age vs. avg_glucose_level")
df.plot(kind='scatter', x='bmi', y='avg_glucose_level', alpha=0.5, color='red', ax=axes[1], title="bmi vs. avg_glucose_level")
plt.show()

* Avg glucose level is high with old people 
* BMI >40 have low avg glucose

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke Or not Pair Grid</h1>

In [8]:
sns.set(style="ticks");
pal = ["#FA5858", "#58D3F7"]

sns.pairplot(df, hue="stroke", palette=pal);
plt.title("stroke");

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Heatmap Correlation</h1>

In [9]:
plt.figure(figsize=(15,7))
sns.heatmap(df.corr(),annot=True);

* There is no correlation between stroke and BMI 

<h1 style="font-family:Georgia;font-size:175%;text-align:left">BMI Boxplot</h1>

In [10]:
plt.figure(figsize=(10,7))
sns.boxplot(data=df,x=df["bmi"],color='green');

In [11]:
bmi_outliers=df.loc[df['bmi']>50]
bmi_outliers['bmi'].shape

In [12]:
# mean with outliers 
print(bmi_outliers['stroke'].value_counts())

In [13]:
df["bmi"] = df["bmi"].apply(lambda x: 50 if x>50 else x)
df["bmi"] = df["bmi"].fillna(28.4)

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Double Check for missing values</h1>

In [14]:
print ("\nMissing values :  ", df.isnull().sum().values.sum())

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke or not in Categorical Features</h1>

In [15]:
cat_df = df[['gender','Residence_type','smoking_status','stroke']]
summary = pd.concat([pd.crosstab(cat_df[x], cat_df.stroke) for x in cat_df.columns[:-1]], keys=cat_df.columns[:-1])
summary

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke/Ever Married</h1>

In [16]:
plt.figure(figsize=(10,5))
strok=df.loc[df['stroke']==1]
sns.countplot(data=strok,x='ever_married',palette='inferno');

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke/Work Type</h1>

In [17]:
plt.figure(figsize=(10,5))
sns.countplot(data=strok,x='work_type',palette='cool');

* Private work exposes you to more stroke

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke/Smoking Status</h1>

In [18]:
plt.figure(figsize=(10,5))
sns.countplot(data=strok,x='smoking_status',palette='autumn');

* Being a smoker or a formerly smoker increases your risk of having a stroke

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Residence Type</h1>

In [19]:
plt.figure(figsize=(10,5))
sns.countplot(data=strok,x='Residence_type',palette='Greens');

* Residence Type has nothing to do with stroke, We cannot take it as a standard


<h1 style="font-family:Georgia;font-size:175%;text-align:left">Stroke/Heart Disease</h1>

In [20]:
plt.figure(figsize=(10,5))
sns.countplot(data=strok,x='heart_disease',palette='Reds');

* Most people who have had a stroke do not have any heart disease, but that does not prevent it being an influential factor 

In [21]:
plt.figure(figsize=(10,5))
sns.countplot(data=strok,x='hypertension',palette='Pastel2');

* more than 25% of strok cases They had hypertension

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Encoding Categorical Features</h1>

In [22]:
df["Residence_type"] = df["Residence_type"].apply(lambda x: 1 if x=="Urban" else 0)
df["ever_married"] = df["ever_married"].apply(lambda x: 1 if x=="Yes" else 0)
df["gender"] = df["gender"].apply(lambda x: 1 if x=="Male" else 0)

 
df = pd.get_dummies(data=df, columns=['smoking_status'])
df = pd.get_dummies(data=df, columns=['work_type'])


In [23]:
df

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Scaling The variance in Features</h1>

In [24]:
std=StandardScaler()
columns = ['avg_glucose_level','bmi','age']
scaled = std.fit_transform(df[['avg_glucose_level','bmi','age']])
scaled = pd.DataFrame(scaled,columns=columns)
df=df.drop(columns=columns,axis=1)

In [25]:
df=df.merge(scaled, left_index=True, right_index=True, how = "left")
df

<h1 style="font-family:Georgia;font-size:175%;text-align:left">Drop ID feature and check for nulls</h1>

In [26]:
df=df.drop(columns='id',axis=1)

In [27]:
df[df.isnull().any(axis=1)]

In [28]:
forest = RandomForestClassifier(n_estimators = 100)
#== 
forest.fit(X_train, y_train)
#== 
#Score 
#== 
forest_score = forest.score(X_train, y_train)
forest_test = forest.score(X_test, y_test)
#== 
#testing model 
#== 
y_pred = forest.predict(X_test)
#== 
#evaluation
#== 
cm = confusion_matrix(y_test,y_pred)
print('Training Score',forest_score)
print('Testing Score \n',forest_test)
print(cm)